### N gram

In [1]:
import re
from nltk.tokenize import word_tokenize
from collections import defaultdict, Counter
from document1 import training_doc1
class MarkovChain:
  def __init__(self):
    self.lookup_dict = defaultdict(list)  
  
  def add_document(self, string):
    preprocessed_list = self._preprocess(string)
    pairs = self.__generate_tuple_keys(preprocessed_list)
    for pair in pairs:
      self.lookup_dict[pair[0]].append(pair[1])
    pairs2 = self.__generate_2tuple_keys(preprocessed_list)
    for pair in pairs2:
      self.lookup_dict[tuple([pair[0], pair[1]])].append(pair[2])
    pairs3 = self.__generate_3tuple_keys(preprocessed_list)
    for pair in pairs3:
      self.lookup_dict[tuple([pair[0], pair[1], pair[2]])].append(pair[3])
  
  def _preprocess(self, string):
    cleaned = re.sub(r'\W+', ' ', string).lower()
    tokenized = word_tokenize(cleaned)
    return tokenized

  def __generate_tuple_keys(self, data):
    if len(data) < 1:
      return

    for i in range(len(data) - 1):
      yield [ data[i], data[i + 1] ]
  
  def __generate_2tuple_keys(self, data):
    if len(data) < 2:
      return

    for i in range(len(data) - 2):
      yield [ data[i], data[i + 1], data[i+2] ]
    
    
  def __generate_3tuple_keys(self, data):
    if len(data) < 3:
      return

    for i in range(len(data) - 3):
      yield [ data[i], data[i + 1], data[i+2], data[i+3] ]
    
  def oneword(self, string):
    return Counter(self.lookup_dict[string]).most_common()[:3]

  def twowords(self, string):
        suggest = Counter(self.lookup_dict[tuple(string)]).most_common()[:3]
        if len(suggest)==0:
            return self.oneword(string[-1])
        return suggest

  def threewords(self, string):
        suggest = Counter(self.lookup_dict[tuple(string)]).most_common()[:3]
        if len(suggest)==0:
            return self.twowords(string[-2:])
        return suggest
    
  def morewords(self, string):
        return self.threewords(string[-3:])

    
  def generate_text(self, string):
    if len(self.lookup_dict) > 0:
        tokens = string.split(" ")
        if len(tokens)==1:
            print("Next word suggestions:", self.oneword(string))
        elif len(tokens)==2:
            print("Next word suggestions:", self.twowords(string.split(" ")))
        elif len(tokens)==3:
            print("Next word suggestions:", self.threewords(string.split(" ")))
        elif len(tokens)>3:
            print("Next word suggestions:", self.morewords(string.split(" ")))
    return

In [4]:
my_markov = MarkovChain()
my_markov.add_document(training_doc1)
my_markov.generate_text(input().lower())

how
Next word suggestions: [('many', 1531), ('did', 459), ('does', 369)]


### NN

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
import numpy as np
import re
from tensorflow.keras.utils import to_categorical
from doc3 import training_doc3

In [7]:
f=open('1661-0.txt',"r",encoding='utf-8')
text = f.read().lower()
print('corpus length:', len(text))

corpus length: 581888


In [19]:
#tokenizer = RegexpTokenizer(r'w+')
#tokens = tokenizer.tokenize(text)

cleaned = re.sub(r'\W+', ' ', text).lower()
tokens = word_tokenize(cleaned)

In [20]:
train_len = 3+1
text_sequences = []

for i in range(train_len,len(tokens)):
    seq = tokens[i-train_len:i]
    text_sequences.append(seq)
sequences = {}
count = 1

for i in range(len(tokens)):
    if tokens[i] not in sequences:
        sequences[tokens[i]] = count
        count += 1
tokenizer = Tokenizer()
tokenizer.fit_on_texts(text_sequences)
sequences = tokenizer.texts_to_sequences(text_sequences) 

#Collecting some information   
vocabulary_size = len(tokenizer.word_counts)+1

n_sequences = np.empty([len(sequences),train_len], dtype='int32')
for i in range(len(sequences)):
    n_sequences[i] = sequences[i]

In [21]:
train_inputs = n_sequences[:,:-1]
train_targets = n_sequences[:,-1]
train_targets = to_categorical(train_targets, num_classes=vocabulary_size)
seq_len = train_inputs.shape[1]
train_inputs.shape
print(train_targets[0])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [22]:
train_targets[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [23]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
#model = load_model("mymodel.h5")

model = Sequential()
model.add(Embedding(vocabulary_size, seq_len, input_length=seq_len))
model.add(LSTM(50,return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50,activation='relu'))
model.add(Dense(vocabulary_size, activation='softmax'))
print(model.summary())
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_inputs,train_targets,epochs=500,verbose=1)
#model.save("mymodel.h5")

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 3, 3)              1365      
_________________________________________________________________
lstm_4 (LSTM)                (None, 3, 50)             10800     
_________________________________________________________________
lstm_5 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 455)               23205     
Total params: 58,120
Trainable params: 58,120
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/500
44/44 [==============================] - 6s 15ms/step - loss: 6.0592 - accuracy: 0.0345
Epoch 2/500

44/44 [==============================] - 1s 12ms/step - loss: 1.2279 - accuracy: 0.6394
Epoch 154/500
44/44 [==============================] - 1s 11ms/step - loss: 1.2271 - accuracy: 0.6394
Epoch 155/500
44/44 [==============================] - 1s 12ms/step - loss: 1.2067 - accuracy: 0.6573
Epoch 156/500
44/44 [==============================] - 1s 12ms/step - loss: 1.1904 - accuracy: 0.6588
Epoch 157/500
44/44 [==============================] - 1s 12ms/step - loss: 1.1844 - accuracy: 0.6580
Epoch 158/500
44/44 [==============================] - 1s 12ms/step - loss: 1.1765 - accuracy: 0.6588
Epoch 159/500
44/44 [==============================] - 0s 11ms/step - loss: 1.1796 - accuracy: 0.6595
Epoch 160/500
44/44 [==============================] - 0s 10ms/step - loss: 1.1695 - accuracy: 0.6602
Epoch 161/500
44/44 [==============================] - 0s 9ms/step - loss: 1.1516 - accuracy: 0.6717
Epoch 162/500
44/44 [==============================] - 0s 8ms/step - loss: 1.1463 - accuracy: 0.6

44/44 [==============================] - 0s 9ms/step - loss: 0.4844 - accuracy: 0.8484
Epoch 314/500
44/44 [==============================] - 1s 12ms/step - loss: 0.4840 - accuracy: 0.8470
Epoch 315/500
44/44 [==============================] - 1s 11ms/step - loss: 0.4682 - accuracy: 0.8513
Epoch 316/500
44/44 [==============================] - 1s 12ms/step - loss: 0.4619 - accuracy: 0.8549
Epoch 317/500
44/44 [==============================] - 0s 10ms/step - loss: 0.4773 - accuracy: 0.8369
Epoch 318/500
44/44 [==============================] - 0s 10ms/step - loss: 0.4650 - accuracy: 0.8585
Epoch 319/500
44/44 [==============================] - 0s 9ms/step - loss: 0.4523 - accuracy: 0.8542
Epoch 320/500
44/44 [==============================] - 0s 9ms/step - loss: 0.4515 - accuracy: 0.8678
Epoch 321/500
44/44 [==============================] - 0s 10ms/step - loss: 0.4518 - accuracy: 0.8585
Epoch 322/500
44/44 [==============================] - 0s 9ms/step - loss: 0.4449 - accuracy: 0.859

Epoch 394/500
44/44 [==============================] - 0s 8ms/step - loss: 0.3966 - accuracy: 0.8649
Epoch 395/500
44/44 [==============================] - 0s 8ms/step - loss: 0.4121 - accuracy: 0.8621
Epoch 396/500
44/44 [==============================] - 0s 8ms/step - loss: 0.4207 - accuracy: 0.8671
Epoch 397/500
44/44 [==============================] - 0s 9ms/step - loss: 0.3906 - accuracy: 0.8728
Epoch 398/500
44/44 [==============================] - 0s 10ms/step - loss: 0.3650 - accuracy: 0.8829
Epoch 399/500
44/44 [==============================] - 0s 9ms/step - loss: 0.3865 - accuracy: 0.8786
Epoch 400/500
44/44 [==============================] - 0s 9ms/step - loss: 0.3581 - accuracy: 0.8858
Epoch 401/500
44/44 [==============================] - 0s 8ms/step - loss: 0.3473 - accuracy: 0.8915
Epoch 402/500
44/44 [==============================] - 0s 9ms/step - loss: 0.3356 - accuracy: 0.8951
Epoch 403/500
44/44 [==============================] - 0s 9ms/step - loss: 0.3280 - accura

In [25]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
input_text = input().strip().lower()
encoded_text = tokenizer.texts_to_sequences([input_text])[0]
pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
print(encoded_text, pad_encoded)
for i in (model.predict(pad_encoded)[0]).argsort()[-3:][::-1]:
  pred_word = tokenizer.index_word[i]
  print("Next word suggestion:",pred_word)

how many
[6, 4] [[0 6 4]]
Next word suggestion: 9
Next word suggestion: city
Next word suggestion: how
